In [ ]:
class ActiveModel(nn.Module):
    def __init__(self, args, input_size, output_size):
        super(ActiveModel, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, output_size)
        self.args = args
        if args.initialize:
            weight_init_factor = args.weight_init_factor
            self.fc1.weight.data.fill_(1) * weight_init_factor
            self.fc2.weight.data.fill_(1) * weight_init_factor
            self.fc1.bias.data.fill_(0)
            self.fc2.bias.data.fill_(0)

    def add_noise_to_grad(self, grad):
        if self.args.gradient_defense:
            noise = torch.randn_like(grad) * self.args.gradient_defense_noise_level
            return grad + noise
        return grad

    def forward(self, x):
        if self.training and self.args.gradient_defense_noise_level:
            x.register_hook(self.add_noise_to_grad)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x